#### @prompt example

## LangChain with OpenBB
This notebook will leverage the following technologies/frameworks to do some stock research.
It will create an Agent that will fetch information for top performing companies in the top performing industry
It will then leverage Chroma to load the latest earning transcript call for Altria (MO) to allow users to ask few questions
on Altria's latest result.
This work was inspired by this article

- Gemini
- LangChain
- OpenBB (openbb.co)

In [2]:
!pip install langchain
!pip install -U langchain-google-genai
!pip install -U -q "google-genai==1.7.0"
!pip install langchain_community
!pip install docx2txt
!pip install chromadb
!pip install wikipedia
!pip install finvizfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigtable 2.27.0 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.
google-generativeai 0.8.3 requires google-ai-generativelanguage==0.6.10, but you have google-ai-generativelanguage 0.6.17 which is incompatible.
opentelemetry-proto 1.32.0 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
pandas-gbq 0.25.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0,

In [3]:
import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [16]:
from langchain_core.tools import tool

@tool
def get_sector_performance():
    """ Return performance by industry for last week, last month, last quarter, last half year and last year"""
    
    from finvizfinance.group import Performance

    try:
        performance = Performance()
        # Get the performance data
        return performance.screener_view().to_dict('records')
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure the finvizfinance library is installed correctly.")
        print("You can install it using: pip install finvizfinance")
        print("Also, check your internet connection as the library fetches data from Finviz.")
def get_companies_for_sector(sectorName:str):
    """ Return a subset of companies for the given sector"""
    from finvizfinance.screener.overview import Overview
    foverview = Overview()
    filters_dict = {'Sector': sectorName,
                   'Market Cap.': '+Mid (over $2bln)',
                    'Current Ratio': 'Over 1',
                    'Debt/Equity': 'Under 1',
                    'P/E': 'Under 10',
                    'InstitutionalOwnership': 'Under 60%',
                    'Price': 'Over $10'}
    foverview.set_filter(filters_dict=filters_dict)
    df = foverview.screener_view(order='Company')
    return df.head(5).to_records()


get_companies_for_sector('Basic Materials')

rec.array([(0, 'BVN', 'Compania de Minas Buenaventura S.A. ADR', 'Basic Materials', 'Other Precious Metals & Mining', 'Peru', 3.770e+09, 9.37, 14.85,  0.0607, 6.233590e+05),
           (1, 'SIM', 'Grupo Simec S.A.B. de C.V. ADR', 'Basic Materials', 'Steel', 'Mexico', 4.280e+09, 7.08, 25.81, -0.0015, 6.060000e+02),
           (2, 'RIO', 'Rio Tinto plc ADR', 'Basic Materials', 'Other Industrial Metals & Mining', 'United Kingdom', 7.163e+10, 8.12, 57.13,  0.0412, 2.537483e+06)],
          dtype=[('index', '<i8'), ('Ticker', 'O'), ('Company', 'O'), ('Sector', 'O'), ('Industry', 'O'), ('Country', 'O'), ('Market Cap', '<f8'), ('P/E', '<f8'), ('Price', '<f8'), ('Change', '<f8'), ('Volume', '<f8')])

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Initialize Gemini Pro model
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7
)

In [12]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

from langchain.agents import create_structured_chat_agent, AgentExecutor

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)



<ipython-input-12-50c6e8cfd421>:21: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()
<ipython-input-12-50c6e8cfd421>:19: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


### Creating tools we will need for our stock research

In [ ]:
@tool
def get_industry_performance() -> list:
    """ Return performance by industry for last week, last month, last quarter, last half year and last year"""
    return obb.equity.compare.groups(group='industry', metric='performance').to_llm()

@tool
def get_strong_buy_for_sector(sector : str) -> list :
    """ Return the strong buy recommendation for a given sector"""
    new_sector = '_'.join(sector.lower().split())
    data = obb.equity.screener(provider='finviz', sector=new_sector, recommendation='buy')
    return data.to_llm()

@tool
def get_best_stock_performers_for_sector(sector:str) -> list :
    """ Return the best  10 stock performers for last week and last month for a sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : sector, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=10)
    return data.to_llm()

@tool
def get_best_stock_performers_for_industry(industry:str) -> list :
    """ Return the best  10 stock performers for last week and last month for a sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : industry, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=10)
    return data.to_llm()

@tool
def get_valuation_for_sectors(input:str) -> list:
    """ Return valuation metrics for the sector provided as input"""
    data = obb.equity.compare.groups(group='sector', metric='valuation', provider='finviz').to_df()
    
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )

@tool
def get_valuation_for_industries(input:str) -> list:
    """ Return valuation meetrics for the industry provided as input"""
    data =  obb.equity.compare.groups(group='industry', metric='valuation', provider='finviz').to_df()
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )
@tool
def get_income_statement(ticker:str) -> list:
    """ Return the last 3  income statement for  a given ticker. Limiting the fields returned to avoid max token errors """
    income_stmnts = obb.equity.fundamental.income(symbol=ticker, limit=3, provider='yfinance').to_df()
    return income_stmnts[['period_ending', 'operating_revenue','total_revenue', 'cost_of_revenue', 'gross_profit']]\
                .to_json(orient="records", date_format="iso", date_unit="s")


@tool
def get_valuation_for_company(ticker:str) -> list:
    """ Return Valuation Ratios  for a company"""#
    # requires obb login
    obb.account.login(pat=os.environ['PAT_KEY'])
    return obb.equity.fundamental.ratios_ttm(symbol='AAPL', provider='fmp', limit=1).to_llm()

#get_strong_buy_for_sector.invoke('Consumer Cyclical')
get_industry_performance()

### Chat Memory

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentType, initialize_agent




MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock recommendation assistant , but dont know current events so you should use your tools as much as you can.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7, verbose=True)
tools = [get_industry_performance]
#llm_with_tools = llm.bind_tools(tools)


chat_history = []
chat_history.append(HumanMessage(content="Your question here"))
chat_history.append(AIMessage(content="AI response here"))
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=17000,
    memory_key="chat_history",
    return_messages=True
)

# Define your prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="chat_history"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        #"chat_history": lambda x: memory.load_memory_variables(x)["chat_history"],
    }
)
'''
agent = create_structured_chat_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)
'''

In [ ]:
agent_executor = initialize_agent(
    llm=llm,
    tools=tools,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
input1 = '''Find the industry that has shown a constant postive performance across quarter, month and week.
'''
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])

In [ ]:
input1 = "Now find me valuation metrics for this sector, please use the right function for this"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])

In [ ]:
!pip install -U langchain-google-genai

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "Which stock are recommented for this sector best performing sector", "chat_history": chat_history})

In [ ]:
obb.equity.fundamental.income(symbol='AAPL', limit=3, provider='yfinance').to_df()